## Feature Engineering

Goal: read in data, clean data, add new features  

Input: raw data  
 
Output:   
1) All data tranformed with new features  
2) Subset of (1) with only Germany data  
3) Subset of (1) with only Ground transit data

### Initial columns

**Quantitative Measures:**  
units_per_order	
transit_days	
deadline_source	
deadline_make	
deadline_deliver

**Categorical:**  
order_id  
country  
shipping_method  
facility  
product_category  
on_sale  
returned  
backorder  
delivered_to_plan

**Duration / Time measures:**
datetime_ordered   
datetime_sourced      
datetime_product_ready     
datetime_planned          
datetime_delivered  


## Imports

In [51]:
# imports
import os
import numpy as np
import pandas as pd
import calendar

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tools.plotting import autocorrelation_plot
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


## Read in data

In [81]:
# read data
def readData():
    path = os.path.join('..','CASE STUDY_DATA_LATE DELIVERY ROOT CAUSE.xlsx')
    df = pd.read_excel(path)

    return df

In [82]:
df = readData()

In [83]:
df.head()

,order_id,country,shipping_method,units_per_order,facility,product_category,on_sale,transit_days,returned,backorder,datetime_ordered,datetime_sourced,datetime_product_ready,datetime_planned,datetime_delivered,deadline_source,deadline_make,deadline_deliver,delivered_to_plan
0,E00000001,UNITED KINGDOM,Ground,1,OXFORD,ACCESSORIES,Y,2,NaN,NaN,2016-07-03 03:07:29,2016-07-03 04:09:49,2016-07-06 00:59:42,2016-07-08,2016-07-07,1612,38,3,PASS
1,E00000002,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,N,3,Y,NaN,2016-07-03 00:08:43,2016-07-03 02:16:24,2016-07-03 07:17:04,2016-07-06,2016-07-05,531,8,3,PASS
2,E00000003,FRANCE,Ground,1,ANTWERP,TOPS,Y,3,NaN,NaN,2016-07-03 00:36:00,2016-07-03 02:16:18,2016-07-03 06:16:57,2016-07-06,2016-07-05,504,8,3,PASS
3,E00000004,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,Y,5,NaN,NaN,2016-07-03 00:47:45,2016-07-03 02:16:18,2016-07-03 06:16:55,2016-07-07,2016-07-05,492,8,5,PASS
4,E00000005,UNITED KINGDOM,Next Day,1,OXFORD,JACKETS & VESTS,Y,1,NaN,NaN,2016-07-03 03:52:13,2016-07-03 07:56:33,2016-07-05 08:53:19,2016-07-06,2016-07-06,1567,6,1,PASS


## Take initial sample if needed
- This was initially used to test quickly with a small amount data.

In [84]:
print (df.shape)

df_sample = df.sample(frac=0.3)

print (df_sample.shape)

(110114, 19)
(33034, 19)


## Set NA as "N" and drop OrderID

In [85]:
def cleanCols(df):
    nan_cols = ['returned','backorder']
    df[nan_cols] = df[nan_cols].fillna('N')
    
    # make cateogrical columns
    cat_cols = df.select_dtypes(include=['object']).columns
    df[cat_cols] = df[cat_cols].apply(lambda x: x.astype('category')).copy()
    
    return df

In [86]:
df = cleanCols(df)
df.drop(['order_id'], axis = 1, inplace=True)

## Make Durations
- source_dur : sourced - planned, minutes
- prod_ready_dur : prod_ready - source, hours
- deliver_dur : deliver - prod_ready, days

In [57]:
df['source_dur'] = (df['datetime_sourced'] - df['datetime_ordered']).apply(lambda x: x.seconds  / (60))
df['prod_ready_dur'] =  (df['datetime_product_ready'] - df['datetime_sourced']).apply(lambda x: x.seconds  / (60 * 60 ))
df['deliver_dur'] =  (df['datetime_delivered'] - df['datetime_product_ready']).apply(lambda x: x.seconds  / (60 * 60 * 24 ))

df['planned_diff'] = (df['datetime_delivered'] - df['datetime_planned']).apply(lambda x: x.seconds  / (60 * 60 * 24 ))

In [58]:
df.head()

,country,shipping_method,units_per_order,facility,product_category,on_sale,transit_days,returned,backorder,datetime_ordered,...,datetime_planned,datetime_delivered,deadline_source,deadline_make,deadline_deliver,delivered_to_plan,source_dur,prod_ready_dur,deliver_dur,planned_diff
0,UNITED KINGDOM,Ground,1,OXFORD,ACCESSORIES,Y,2,N,N,2016-07-03 03:07:29,...,2016-07-08,2016-07-07,1612,38,3,PASS,62.333333,20.831389,0.958542,0.0
1,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,N,3,Y,N,2016-07-03 00:08:43,...,2016-07-06,2016-07-05,531,8,3,PASS,127.683333,5.011111,0.696481,0.0
2,FRANCE,Ground,1,ANTWERP,TOPS,Y,3,N,N,2016-07-03 00:36:00,...,2016-07-06,2016-07-05,504,8,3,PASS,100.300000,4.010833,0.738229,0.0
3,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,Y,5,N,N,2016-07-03 00:47:45,...,2016-07-07,2016-07-05,492,8,5,PASS,88.550000,4.010278,0.738252,0.0
4,UNITED KINGDOM,Next Day,1,OXFORD,JACKETS & VESTS,Y,1,N,N,2016-07-03 03:52:13,...,2016-07-06,2016-07-06,1567,6,1,PASS,244.333333,0.946111,0.629641,0.0


## Make Duration Proportions
- source_dur_prop : sourcing duration / total duration
- prod_ready_dur_prop : make duration / total duration
- deliver_dur_prop : delivery duration / total duration
- total_dur : sum of all durations  (in seconds)

In [59]:
df['total_dur'] = (df['datetime_delivered'] - df['datetime_ordered']).apply(lambda x: x.seconds)

df['source_dur_prop'] =  (df['datetime_sourced'] - df['datetime_ordered']).apply(lambda x: x.seconds) / df['total_dur'] 
df['prod_ready_dur_prop'] =  (df['datetime_product_ready'] - df['datetime_sourced']).apply(lambda x: x.seconds) / df['total_dur'] 
df['deliver_dur_prop'] =  (df['datetime_delivered'] - df['datetime_product_ready']).apply(lambda x: x.seconds) / df['total_dur'] 

In [60]:
df.head()

,country,shipping_method,units_per_order,facility,product_category,on_sale,transit_days,returned,backorder,datetime_ordered,...,deadline_deliver,delivered_to_plan,source_dur,prod_ready_dur,deliver_dur,planned_diff,total_dur,source_dur_prop,prod_ready_dur_prop,deliver_dur_prop
0,UNITED KINGDOM,Ground,1,OXFORD,ACCESSORIES,Y,2,N,N,2016-07-03 03:07:29,...,3,PASS,62.333333,20.831389,0.958542,0.0,75151,0.049766,0.997898,1.102021
1,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,N,3,Y,N,2016-07-03 00:08:43,...,3,PASS,127.683333,5.011111,0.696481,0.0,85877,0.089209,0.210068,0.700723
2,FRANCE,Ground,1,ANTWERP,TOPS,Y,3,N,N,2016-07-03 00:36:00,...,3,PASS,100.300000,4.010833,0.738229,0.0,84240,0.071439,0.171403,0.757158
3,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,Y,5,N,N,2016-07-03 00:47:45,...,5,PASS,88.550000,4.010278,0.738252,0.0,83535,0.063602,0.172826,0.763572
4,UNITED KINGDOM,Next Day,1,OXFORD,JACKETS & VESTS,Y,1,N,N,2016-07-03 03:52:13,...,1,PASS,244.333333,0.946111,0.629641,0.0,72467,0.202299,0.047001,0.750700


## Extract day and month

New columns:
- Order month
- Order day of the week
- Delivery day of the week

In [61]:
df['order_month'] = df['datetime_ordered'].apply(lambda x: calendar.month_name[x.month])
df['order_day'] = df['datetime_ordered'].apply(lambda x: calendar.day_name[x.weekday()])
df['deliver_day'] = df['datetime_delivered'].apply(lambda x: calendar.day_name[x.weekday()])

In [62]:
df.head()

,country,shipping_method,units_per_order,facility,product_category,on_sale,transit_days,returned,backorder,datetime_ordered,...,prod_ready_dur,deliver_dur,planned_diff,total_dur,source_dur_prop,prod_ready_dur_prop,deliver_dur_prop,order_month,order_day,deliver_day
0,UNITED KINGDOM,Ground,1,OXFORD,ACCESSORIES,Y,2,N,N,2016-07-03 03:07:29,...,20.831389,0.958542,0.0,75151,0.049766,0.997898,1.102021,July,Sunday,Thursday
1,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,N,3,Y,N,2016-07-03 00:08:43,...,5.011111,0.696481,0.0,85877,0.089209,0.210068,0.700723,July,Sunday,Tuesday
2,FRANCE,Ground,1,ANTWERP,TOPS,Y,3,N,N,2016-07-03 00:36:00,...,4.010833,0.738229,0.0,84240,0.071439,0.171403,0.757158,July,Sunday,Tuesday
3,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,Y,5,N,N,2016-07-03 00:47:45,...,4.010278,0.738252,0.0,83535,0.063602,0.172826,0.763572,July,Sunday,Tuesday
4,UNITED KINGDOM,Next Day,1,OXFORD,JACKETS & VESTS,Y,1,N,N,2016-07-03 03:52:13,...,0.946111,0.629641,0.0,72467,0.202299,0.047001,0.750700,July,Sunday,Wednesday


## Days since last failed delivery by country and facility


In [63]:
def lastFcltyFail(row):
    facility = row['facility']
    time = row['datetime_delivered']
    
    last_date = np.max(df_fails[(df_fails['facility'] == facility) 
                          & (df_fails['datetime_delivered'] < time)]['datetime_delivered'])
    diff = time - last_date
    
    if pd.isnull(diff):
        return 0
    else:
        return (diff.seconds  / (60 * 60 * 24 ))

In [64]:
def lastCountryFail(row):
    country = row['country']
    time = row['datetime_delivered']
    
    last_date = np.max(df_fails[(df_fails['country'] == country) 
                          & (df_fails['datetime_delivered'] < time)]['datetime_delivered'])
    diff = time - last_date
    
    if pd.isnull(diff):
        return 0
    else:
        return (diff.seconds  / (60 * 60 * 24 ))
    

In [65]:
# create a smaller subset of data to improve performance when joining data
df_fails = df[df['delivered_to_plan'] == 'FAIL'][['country','facility','datetime_delivered']]

df['days_since_fclty_fail'] = df.apply(axis=1, func=lastFcltyFail)
df['days_since_country_fail'] = df.apply(axis=1, func=lastCountryFail)

In [66]:
df.head()

,country,shipping_method,units_per_order,facility,product_category,on_sale,transit_days,returned,backorder,datetime_ordered,...,planned_diff,total_dur,source_dur_prop,prod_ready_dur_prop,deliver_dur_prop,order_month,order_day,deliver_day,days_since_fclty_fail,days_since_country_fail
0,UNITED KINGDOM,Ground,1,OXFORD,ACCESSORIES,Y,2,N,N,2016-07-03 03:07:29,...,0.0,75151,0.049766,0.997898,1.102021,July,Sunday,Thursday,0.0,0.0
1,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,N,3,Y,N,2016-07-03 00:08:43,...,0.0,85877,0.089209,0.210068,0.700723,July,Sunday,Tuesday,0.0,0.0
2,FRANCE,Ground,1,ANTWERP,TOPS,Y,3,N,N,2016-07-03 00:36:00,...,0.0,84240,0.071439,0.171403,0.757158,July,Sunday,Tuesday,0.0,0.0
3,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,Y,5,N,N,2016-07-03 00:47:45,...,0.0,83535,0.063602,0.172826,0.763572,July,Sunday,Tuesday,0.0,0.0
4,UNITED KINGDOM,Next Day,1,OXFORD,JACKETS & VESTS,Y,1,N,N,2016-07-03 03:52:13,...,0.0,72467,0.202299,0.047001,0.750700,July,Sunday,Wednesday,0.0,0.0


## Supply chain week overlap
- Check if phase transitions happen in the same week
- Get the number of weeks in between each phase (same week would be 0, next week would be 1) for each phase


In [67]:
df['source_week'] = df['datetime_sourced'].apply(lambda x: x.week) - df['datetime_ordered'].apply(lambda x: x.week)
df['prod_ready_week'] =  df['datetime_product_ready'].apply(lambda x: x.week) - df['datetime_sourced'].apply(lambda x: x.week)
df['deliver_week'] =  df['datetime_delivered'].apply(lambda x: x.week) - df['datetime_product_ready'].apply(lambda x: x.week)

In [68]:
df.head()

,country,shipping_method,units_per_order,facility,product_category,on_sale,transit_days,returned,backorder,datetime_ordered,...,prod_ready_dur_prop,deliver_dur_prop,order_month,order_day,deliver_day,days_since_fclty_fail,days_since_country_fail,source_week,prod_ready_week,deliver_week
0,UNITED KINGDOM,Ground,1,OXFORD,ACCESSORIES,Y,2,N,N,2016-07-03 03:07:29,...,0.997898,1.102021,July,Sunday,Thursday,0.0,0.0,0,1,0
1,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,N,3,Y,N,2016-07-03 00:08:43,...,0.210068,0.700723,July,Sunday,Tuesday,0.0,0.0,0,0,1
2,FRANCE,Ground,1,ANTWERP,TOPS,Y,3,N,N,2016-07-03 00:36:00,...,0.171403,0.757158,July,Sunday,Tuesday,0.0,0.0,0,0,1
3,FRANCE,Ground,1,ANTWERP,JACKETS & VESTS,Y,5,N,N,2016-07-03 00:47:45,...,0.172826,0.763572,July,Sunday,Tuesday,0.0,0.0,0,0,1
4,UNITED KINGDOM,Next Day,1,OXFORD,JACKETS & VESTS,Y,1,N,N,2016-07-03 03:52:13,...,0.047001,0.750700,July,Sunday,Wednesday,0.0,0.0,0,1,0


## Make dummy variables

- drop date variables as now included within durations and calendar columns
- randomly subset the dataframe to use for modeling (performance reasons)

In [71]:
date_cols = df.select_dtypes(include=['datetime64[ns]']).columns
df.drop(date_cols, axis=1, inplace=True)

In [73]:
df_dummies = pd.get_dummies(df).copy()

In [74]:
df_dummies.head()

,units_per_order,transit_days,deadline_source,deadline_make,deadline_deliver,source_dur,prod_ready_dur,deliver_dur,planned_diff,total_dur,...,order_day_Thursday,order_day_Tuesday,order_day_Wednesday,deliver_day_Friday,deliver_day_Monday,deliver_day_Saturday,deliver_day_Sunday,deliver_day_Thursday,deliver_day_Tuesday,deliver_day_Wednesday
0,1,2,1612,38,3,62.333333,20.831389,0.958542,0.0,75151,...,0,0,0,0,0,0,0,1,0,0
1,1,3,531,8,3,127.683333,5.011111,0.696481,0.0,85877,...,0,0,0,0,0,0,0,0,1,0
2,1,3,504,8,3,100.300000,4.010833,0.738229,0.0,84240,...,0,0,0,0,0,0,0,0,1,0
3,1,5,492,8,5,88.550000,4.010278,0.738252,0.0,83535,...,0,0,0,0,0,0,0,0,1,0
4,1,1,1567,6,1,244.333333,0.946111,0.629641,0.0,72467,...,0,0,0,0,0,0,0,0,0,1


In [75]:
df_dummies.dtypes

units_per_order                       int64
transit_days                          int64
deadline_source                       int64
deadline_make                         int64
deadline_deliver                      int64
source_dur                          float64
prod_ready_dur                      float64
deliver_dur                         float64
planned_diff                        float64
total_dur                             int64
source_dur_prop                     float64
prod_ready_dur_prop                 float64
deliver_dur_prop                    float64
days_since_fclty_fail               float64
days_since_country_fail             float64
source_week                           int64
prod_ready_week                       int64
deliver_week                          int64
country_BELGIUM                       uint8
country_FRANCE                        uint8
country_GERMANY                       uint8
country_NETHERLANDS                   uint8
country_SWEDEN                  

## Save Data - All Data

In [76]:
file = os.path.join('..','data_transformed_sample.csv')
df_dummies.to_csv(file, index=False)

## Germany only data

In [77]:
germany = df_dummies[df_dummies['country_GERMANY'] == 1]
germany.shape

(61764, 65)

In [78]:
file = os.path.join('..','germany.csv')
germany.to_csv(file, index=False)

## Ground transport only data

In [79]:
ground = df_dummies[df_dummies['shipping_method_Ground'] == 1]
ground.shape

(76821, 65)

In [80]:
file = os.path.join('..','ground.csv')
ground.to_csv(file, index=False)